In [1]:
from scipy import stats
import numpy as np
import math

# Задание 1

Применяем гипотезу о среднем

 - H0 - среднее занчение ряда равно u0
 - H1 - среднее значение ряда не равно u0 (больше или меньше)

In [2]:
# Данные для проверки
arr = np.array([12.9, 11.6, 13.5, 13.9, 12.1, 11.9, 13.0])

In [3]:
# среднее значение
u0 = 12

### Проверка распределения на нормальность

In [5]:
# Проверяем данные на нормальность, используя критерий Шапиро-Уилка
stats.shapiro(arr)

ShapiroResult(statistic=0.9461528332649155, pvalue=0.694601251101445)

На уровне значимости alpha = 0.05 принимаем, что данные arr распределены нормально (т.к. p-value = 0.69 > 0.05)

### Применяем ttest_1samp

 - Условие нормальности распределения данных выполняется, можем применять другие критерии
 - Применяем t-test для одного сэмла данных (у нас всего один ряд)
 - Данный тест с двусторонней критичекой областью (нам такой и нужен, он учитывает отклонение в обе стороны)

In [6]:
alpha = 0.05
p = stats.ttest_1samp(arr, u0)
p

TtestResult(statistic=2.1578017974978794, pvalue=0.07430190210562303, df=6)

В качестве уровня значимости принимаем alpha = 0.05

In [7]:
print('p-value = ', p.pvalue)
print(alpha > p.pvalue)

p-value =  0.07430190210562303
False


Значение alpha меньше pvalue, поэтому мы не можем отвергнуть нулевую гипотезу и принимаем, что значение 12 действительно является номинальным напряжением

### Пересчет руками

Посчитаем то же самое руками

In [8]:
u = arr.mean() # считаем среднее значение
print(u)

12.700000000000001


In [9]:
T = (u0 - u) * math.sqrt(len(arr) - 1) / np.std(arr) 
T # считаем статистику критерия

-2.1578017974978794

In [11]:
# Переведем значение статистики критерия в p-value

krit = stats.t(len(arr) - 1)
p = krit.cdf(T) * 2
print(p) # p-value для двустороннего теста

0.07430190210562303


# Задание 2

 - Так же применяем гипотезу о среднем, но для парных наблюдений
 - Выборки являются зависимыми (один и тот же образец измерен двумя методами)

In [48]:
# нужно анализировать разность рядов и по Н0 сравнивать с 0

a = np.array([23.1, 23.2, 26.5, 26.6, 27.1, 48.3, 40.5, 25.0, 38.4, 23.5]) # первый метод
b = np.array([22.7, 23.6, 27.1, 27.4, 27.4, 46.8, 40.4, 24.9, 38.1, 23.8]) # второй метод

### Проверка на нормальность распределения

In [51]:
print(stats.shapiro(a).pvalue)
print(stats.shapiro(b).pvalue)

0.009897167254532586
0.02066336811509381


Сами выборки распределены ненормально на уровне значимости 0.05

### Разность парных данных в ttest_1samp 

 - H0 - разница средних для первого и второго ряда равна 0 или m1 - m2 = 0
 - H1 - разница средних не равна 0 в любую сторону или m1 != m2

In [17]:
c = a - b
print(c)

[ 0.4 -0.4 -0.6 -0.8 -0.3  1.5  0.1  0.1  0.3 -0.3]


Проверяем разность данных на нормальность с помощью критерия Шапиро-Уилка

In [18]:
stats.shapiro(c)

ShapiroResult(statistic=0.8998483579792755, pvalue=0.21825780478589868)

Данные разности являются нормальными на уровне значимости 0.05

 - Выполняется условие независимости данных
 - Можем использовать тот же t-критерий, что и в предыдущем задании
 - Критерий двусторонний, что соответствует заявленной альтернативной гипотезе

In [19]:
p = stats.ttest_1samp(c, 0)
p

TtestResult(statistic=1.7154884978678027e-15, pvalue=0.9999999999999987, df=9)

In [20]:
alpha = 0.05
print('p-value = ', p.pvalue)
print(alpha > p.pvalue)

p-value =  0.9999999999999987
False


Значение alpha меньше pvalue, поэтому мы не можем отвергнуть нулевую гипотезу и принимаем, что средние значения показаний обоих методов равны

### Встроенный парный тест ttest_rel

 - В библиотеке есть реализованный тест для парных зависимых выборок,
пробуем применить его
 - тест параметрический, для интереса пробуем применить его, несмотря на ненормальность распределения изначальных выборок

In [50]:
stats.ttest_rel(a, b)
# тот же самый результат

TtestResult(statistic=1.7154884978678027e-15, pvalue=0.9999999999999987, df=9)

### Пересчет руками

Пробуем считать руками то же самое руками

In [24]:
# Считаем значение статистики критерия
T = (0 - c.mean()) * (len(c) - 1) ** 0.5 / c.std()
T

-1.715488497867803e-15

In [26]:
tt = stats.t(len(c) - 1)
p = tt.cdf(T) * 2
p

0.9999999999999987

### Тест Уилкоксона

 - А теперь примем во внимание ненормальность распределения выборок и применим непараметрический тест
 - Выбираем тест Уилкоксона

In [54]:
stats.wilcoxon(a, y=b)

WilcoxonResult(statistic=23.0, pvalue=0.6953125)

Даже в этом случае на уровне значимости 0.05 разницы в методах измерения не наблюдается

# Задание 3

### Проверка на нормальность распределения

Проверка данных на нормальность распределения с помощью критерия Шапиро-Уилка

In [28]:
city = np.array([132, 111, 119, 138, 200, 131, 138, 170, 159, 140])
vil = np.array([115, 190, 127, 155, 148, 121, 116, 121, 197])

In [29]:
stats.shapiro(city)
# Данные распределены нормально

ShapiroResult(statistic=0.9100654196723891, pvalue=0.281440686937816)

In [46]:
stats.shapiro(vil)
# Данные распределены ненормально
# Возможно такие результаты наблюдаются из-за малого количества данных во второй выборке

ShapiroResult(statistic=0.8286640993349573, pvalue=0.04313951145476073)

### Попытка в параметрические тесты ttest_ind

 - Не смотря на ложность условия нормальности распределения данных пробуем применить критерий Стьюдента для проверки гипотезы о равенстве средних для независимых выборок (для интереса)
 - Одновременно с ненормальностью распределения второй выборки считаем, что не наблюдаем равенство дисперсий выборок
 - альтернативная гипотеза - односторонняя (давление горожан выше), поэтому добавляем параметр greater в тест

In [65]:
preassure = stats.ttest_ind(city, vil, equal_var=False, alternative='greater') 
alpha = 0.05

In [63]:
preassure

TtestResult(statistic=0.03479498734072727, pvalue=0.4863422498708747, df=15.601702532561607)

Pvalue выше, чем alpha, поэтому принимаем нулевую гипотезу о равенстве средних

### Критерий Манна-Уитни

 - В силу ненормальности распределения второй выборки пробуем применить непараметический тест
 - Выбираем критерий Маннf-Уитни 

In [64]:
stats.mannwhitneyu(city, vil, alternative='greater')

MannwhitneyuResult(statistic=50.0, pvalue=0.3565312487796153)

Полученное в непараметрическом тесте pvalue не позволяет отвергнуть нулевую гипотезу.

Считаем средние значения двух выборок неотличными друг от друга.

# Задание 4

 - H0 - соотношение мужчин и женщин в возрасте от 30 до 60 лет такое же, как у новорожденных: p = 0.48 (женщин), соответственно смертность среди мужчин и женщин одинакова
 - H1 - смертность различается (в любую сторону) p != 0.48
 - уровень значимости alpha = 0.05

 - для проверки нулевой гипотезы будем использовать proportion_ztest
 - по умолчанию он двусторонний, так и оставим

In [73]:
import statsmodels.api as sm

In [90]:
sm.stats.proportions_ztest(
    2500,   # количество женщин в возрасте от 30 до 60 (50% от общей выборки)
    5000,   # количество людей в выборке
    0.48    # доля новорожденных для сравнения 
    # alternative = 'larger'
)

(2.8284271247461925, 0.00467773498104723)

p-value меньше, чем alpha, поэтому отвергаем нулевую гипотезу
смертность мужчин и женщин неодинакова (смертность женщин ниже)

# Задание 5

Причины значимости и незначимости различий результатов тестов:
- Размер выборки - в первом случае могла быть достаточно большая выборка, на которой получилось увидеть различие в данных. Во второй выборка могла быть небольшой.
- Наличие выбросов или высокого стандартного отклонения - в первом тесте могло идти сравнение результатов экспериментов наподобие (17.1, 16.9, 17.0) и (16.2, 16.1, 15.7). В таком случае разница достаточна очевидна. А во втором тесте могли быть (-100, 51, 100) и (35, 36, 37) и тест выдал незначимость различий.
- Возможно критерий Стьюдента не подходил для эксперимента (выборки не были независимыми или распределение было не нормальным)